# Segment, localise and track


Fixing a bug in the code that meant the intracellular mean intensity values weren't accurate due to the downscaling

In [1]:
from macrohet import dataio, tile
from macrohet.notify import send_sms
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import os
import dask.array as da
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
# defining cellpose model to use
model = models.Cellpose(
                        gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Define functions to tidy up main block of code

In [2]:
def segment(frame, model = model, channels = [0,0], diameter = 325, min_size = 2500):
    
    masks, flows, styles, diams = model.eval(frame, 
                                             channels = channels, 
                                             diameter = diameter, 
                                             min_size = min_size, 
                                             )
    return masks

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(segmentation=masks,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(1/(5.04),1/(5.04)),
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
                                                   
    return objects

# define config fn to use
config_fn = '/home/dayn/analysis/BayesianTracker/models/cell_config.json'

def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]), (0, masks.shape[-1]))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [3]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [4]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Segment, localise and track

In [5]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')

In [6]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):

    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # reload seg
    tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
    with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
        masks = hdf.segmentation

    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/04/28 11:05:32 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 4)_objs_masks_tracks.h5...
[INFO][2023/04/28 11:05:56 AM] Loading segmentation (75, 6048, 6048)
[INFO][2023/04/28 11:05:56 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 4)_objs_masks_tracks.h5
[INFO][2023/04/28 11:05:56 AM] Localizing objects from segmentation...
 85%|██████████████████████████████████████████████████████████▉          | 64/75 [17:17<02:58, 16.21s/it]

KeyboardInterrupt



In [ ]:
send_sms('Complete retracking job done')

# Check new retracked h5s

In [ ]:
row, column = 3, 5

In [39]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/relocalised_{row, column}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
            tracks_loaded = reader.tracks
            objects_loaded = reader.objects
            segmentation_loaded = reader.segmentation

[INFO][2023/04/28 09:04:57 AM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(5, 9).h5...
[INFO][2023/04/28 09:04:57 AM] Loading tracks/obj_type_1
[INFO][2023/04/28 09:04:57 AM] Loading LBEP/obj_type_1
[INFO][2023/04/28 09:04:57 AM] Loading objects/obj_type_1 (25615, 5) (25615 filtered: None)
[INFO][2023/04/28 09:04:57 AM] Closing HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(5, 9).h5


NameError: free variable 'dummy_obj' referenced before assignment in enclosing scope

In [31]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,area,mean_intensity,major_axis_length,orientation,minor_axis_length
0,324,0,955.750079,26.453723,0.000000,324,324,5,0,False,38263.0,"(2,) array",296.419416,-1.410147,169.649215
1,324,1,961.911381,25.325357,0.000000,324,324,5,0,False,52851.0,"(2,) array",386.590538,-1.535681,194.568067
2,324,2,964.602978,26.337461,0.000000,324,324,5,0,False,52894.0,"(2,) array",330.689148,-1.499841,219.315337
3,324,3,963.317088,22.161148,0.000000,324,324,5,0,False,50287.0,"(2,) array",379.267846,-1.503667,179.383803
4,324,4,963.083443,23.526086,0.000000,324,324,5,0,False,43029.0,"(2,) array",330.320404,-1.380799,180.160613
5,324,5,963.183362,23.756568,0.497968,324,324,5,0,True,NaN,"(2,) array",NaN,NaN,NaN
6,324,6,965.002195,23.610383,0.000000,324,324,5,0,False,45249.0,"(2,) array",396.892406,-1.317333,161.264737
7,324,7,957.956495,28.067998,0.000000,324,324,5,0,False,52266.0,"(2,) array",442.716850,-1.388103,161.996001
8,324,8,958.539196,27.097019,0.000000,324,324,5,0,False,52387.0,"(2,) array",485.450252,-1.291636,149.430947
9,324,9,962.485546,25.507084,0.000000,324,324,5,0,False,51945.0,"(2,) array",491.030363,-1.242747,144.363657


In [44]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/test2.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_tracks(tracks)
#             writer.write_objects(objects)


[INFO][2023/04/28 09:06:24 AM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/test2.h5...
[INFO][2023/04/28 09:06:24 AM] Writing objects/obj_type_1
[INFO][2023/04/28 09:06:24 AM] Writing labels/obj_type_1
[INFO][2023/04/28 09:06:24 AM] Loading objects/obj_type_1 (25514, 5) (25514 filtered: None)
[INFO][2023/04/28 09:06:25 AM] Writing properties/obj_type_1/area (25514,)
[INFO][2023/04/28 09:06:25 AM] Writing properties/obj_type_1/major_axis_length (25514,)
[INFO][2023/04/28 09:06:25 AM] Writing properties/obj_type_1/minor_axis_length (25514,)
[INFO][2023/04/28 09:06:25 AM] Writing properties/obj_type_1/orientation (25514,)
[INFO][2023/04/28 09:06:25 AM] Writing properties/obj_type_1/mean_intensity (25514, 2)
[INFO][2023/04/28 09:06:25 AM] Writing tracks/obj_type_1
[INFO][2023/04/28 09:06:25 AM] Writing LBEP/obj_type_1
[INFO][2023/04/28 09:06:25 AM] Writing fates/obj_type_1
[INFO][2023/04/28 09:06:25 AM] Closing HDF file: /mnt/DATA/macrohet/labels/relocalisation/test2.h5


In [45]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/test.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
            tracks_loaded = reader.tracks

[INFO][2023/04/28 09:06:27 AM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/test.h5...
[INFO][2023/04/28 09:06:27 AM] Loading tracks/obj_type_1
[INFO][2023/04/28 09:06:27 AM] Loading LBEP/obj_type_1
[INFO][2023/04/28 09:06:27 AM] Loading objects/obj_type_1 (25514, 5) (25514 filtered: None)
[INFO][2023/04/28 09:06:28 AM] Closing HDF file: /mnt/DATA/macrohet/labels/relocalisation/test.h5


NameError: free variable 'dummy_obj' referenced before assignment in enclosing scope

# Reload btrack with my fix and test

In [56]:
import importlib
importlib.reload(btrack.io.hdf)

<module 'btrack.io.hdf' from '/home/dayn/analysis/btrack/btrack/io/hdf.py'>

In [53]:
row, column = 6, 9

In [58]:
# save out 
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/relocalisation/relocalised_{row, column}.h5'), 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
        writer.write_objects(objects)
        writer.write_tracks(tracks)
        writer.write_segmentation(masks)

[INFO][2023/04/28 09:36:20 AM] Opening HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(6, 9).h5...
[INFO][2023/04/28 09:36:20 AM] Writing objects/obj_type_1
[INFO][2023/04/28 09:36:20 AM] Writing labels/obj_type_1
[INFO][2023/04/28 09:36:20 AM] Closing HDF file: /mnt/DATA/macrohet/labels/relocalisation/relocalised_(6, 9).h5


AssertionError: 

In [47]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,area,mean_intensity,major_axis_length,orientation,minor_axis_length
0,324,0,955.750079,26.453723,0.000000,324,324,5,0,False,38263.0,"(2,) array",296.419416,-1.410147,169.649215
1,324,1,961.911381,25.325357,0.000000,324,324,5,0,False,52851.0,"(2,) array",386.590538,-1.535681,194.568067
2,324,2,964.602978,26.337461,0.000000,324,324,5,0,False,52894.0,"(2,) array",330.689148,-1.499841,219.315337
3,324,3,963.317088,22.161148,0.000000,324,324,5,0,False,50287.0,"(2,) array",379.267846,-1.503667,179.383803
4,324,4,963.083443,23.526086,0.000000,324,324,5,0,False,43029.0,"(2,) array",330.320404,-1.380799,180.160613
5,324,5,963.183362,23.756568,0.497968,324,324,5,0,True,NaN,"(2,) array",NaN,NaN,NaN
6,324,6,965.002195,23.610383,0.000000,324,324,5,0,False,45249.0,"(2,) array",396.892406,-1.317333,161.264737
7,324,7,957.956495,28.067998,0.000000,324,324,5,0,False,52266.0,"(2,) array",442.716850,-1.388103,161.996001
8,324,8,958.539196,27.097019,0.000000,324,324,5,0,False,52387.0,"(2,) array",485.450252,-1.291636,149.430947
9,324,9,962.485546,25.507084,0.000000,324,324,5,0,False,51945.0,"(2,) array",491.030363,-1.242747,144.363657
